In [1]:
import pandas as pd
import numpy as np

In [47]:
rf_train = pd.read_csv('./TrainSubmission/rf_train.csv')
gb_train = pd.read_csv('./TrainSubmission/gb_train.csv')
dt_train = pd.read_csv('./TrainSubmission/dt_final_train.csv')
ridge_train = pd.read_csv('./TrainSubmission/bagging_ridge_train.csv')
svr_train = pd.read_csv('./TrainSubmission/bagging_svr_train.csv')
xgb_train = pd.read_csv('./TrainSubmission/xgb_train.csv')
adb_train = pd.read_csv('./TrainSubmission/adb_train.csv')
ma_train = pd.read_csv('./TrainSubmission/ma_train.csv')

In [48]:
print(rf_train.shape)
print(gb_train.shape)
print(dt_train.shape)
print(ridge_train.shape)
print(svr_train.shape)
print(xgb_train.shape)
print(adb_train.shape)
print(ma_train.shape)

(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)


In [49]:
# merge_train = pd.merge(ma_train, rf_train, on='ID', how='left', suffixes=('_ma', '_rf'))
merge_train = pd.merge(ma_train, gb_train, on='ID', how='left', suffixes=('_ma', '_gb'))
# merge_train = pd.merge(merge_train, dt_train, on='ID', how='left', suffixes=('_gb', '_dt'))
# merge_train = pd.merge(merge_train, ridge_train, on='ID', how='left', suffixes=('_dt', '_ridge'))
# merge_train = pd.merge(merge_train, svr_train, on='ID', how='left', suffixes=('_ridge', '_svr'))
merge_train = pd.merge(merge_train, xgb_train, on='ID', how='left', suffixes=('_gb', '_xgb'))
merge_train = pd.merge(merge_train, adb_train, on='ID', how='left', suffixes=('_xgb', '_adb'))
merge_train['stacked_prediction'] = 0.7 * ma_train['TARGET'] + 0.1 * gb_train['TARGET'] + 0.1 * xgb_train['TARGET'] + 0.1 * adb_train['TARGET']
X = merge_train

In [50]:
y = pd.read_csv('../challenge_data/Y_train.csv')

In [51]:
ma_test = pd.read_csv('./Submission/MA_final.csv')
rf_test = pd.read_csv('./Submission/rf_test.csv')
gb_test = pd.read_csv('./Submission/gb_final.csv')
dt_test = pd.read_csv('./Submission/dt_final.csv')
ridge_test = pd.read_csv('./Submission/bagging_linear_test.csv')
svr_test = pd.read_csv('./Submission/bagging_svr_test.csv')
xgb_test = pd.read_csv('./Submission/xgb_final.csv')
adb_test = pd.read_csv('./Submission/adab_final.csv')

In [52]:
# merge_test = pd.merge(ma_test, rf_test, on='ID', how='left', suffixes=('_ma', '_rf'))
merge_test = pd.merge(ma_test, gb_test, on='ID', how='left', suffixes=('_ma', '_gb'))
# merge_test = pd.merge(merge_test, dt_test, on='ID', how='left', suffixes=('_gb', '_dt'))
# merge_test = pd.merge(merge_test, ridge_test, on='ID', how='left', suffixes=('_dt', '_ridge'))
# merge_test = pd.merge(merge_test, svr_test, on='ID', how='left', suffixes=('_ridge', '_svr'))
merge_test = pd.merge(merge_test, xgb_test, on='ID', how='left', suffixes=('_gb', '_xgb'))
merge_test = pd.merge(merge_test, adb_test, on='ID', how='left', suffixes=('_xgb', '_adb'))
merge_test['stacked_prediction'] = 0.7 * ma_test['TARGET'] + 0.1 * gb_test['TARGET'] + 0.1 * xgb_test['TARGET'] + 0.1 * adb_test['TARGET']
X_test = merge_test

In [53]:
X_test

,ID,TARGET_ma,TARGET_gb,TARGET_xgb,TARGET_adb,stacked_prediction
0,1115,0.047286,0.052237,-0.003880,0.081199,0.046056
1,1202,0.020810,0.031621,0.196285,0.036961,0.041053
2,1194,0.246302,0.059642,0.312551,0.903239,0.299954
3,1084,0.280554,0.052237,-0.034276,0.082675,0.206451
4,1135,0.132439,0.052237,0.190241,0.312328,0.148188
...,...,...,...,...,...,...
649,879,-0.906298,-0.715887,-1.020367,-0.370244,-0.845058
650,673,-0.620182,0.066356,-0.170188,0.072929,-0.437218
651,1641,0.302310,0.274097,1.319926,0.676229,0.438642
652,712,2.131317,0.476056,1.421553,0.509003,1.732583


In [54]:
X = X.set_index('ID').sort_index().reset_index(drop=True)
y = y.set_index('ID').sort_index().reset_index(drop=True)
X_test = X_test.set_index('ID').sort_index().reset_index(drop=True)

In [55]:
print(X.shape)
print(X_test.shape)

(1494, 5)
(654, 5)


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from scipy.stats import spearmanr

In [57]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
model = LinearRegression()
model.fit(x_train, y_train['TARGET'])
y_pred = model.predict(x_test)
spearman_corr_lr = spearmanr(y_pred, y_test).correlation
print(f"Spearman Correlation for Linear Regression: {spearman_corr_lr:.1%}")

Spearman Correlation for Linear Regression: 75.5%


In [61]:
coefficients = model.coef_
feature_names = X.columns
feature_weights = list(zip(feature_names, coefficients))
for feature, weight in feature_weights:
    print(f"{feature}: {weight:.4f}")

TARGET_ma: -0.0430
TARGET_gb: 0.4598
TARGET_xgb: 0.4598
TARGET_adb: 0.1619
stacked_prediction: 0.0781


In [62]:
from sklearn.svm import SVR

svm_model = SVR()
svm_model.fit(x_train, y_train['TARGET'])
y_pred_svm = svm_model.predict(x_test)
spearman_corr_svm = spearmanr(y_pred_svm, y_test).correlation
print(f"Spearman Correlation for SVM: {spearman_corr_svm:.1%}")

Spearman Correlation for SVM: 77.8%


In [63]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(x_train, y_train['TARGET'])
y_pred_rf = rf_model.predict(x_test)
spearman_corr_rf = spearmanr(y_pred_rf, y_test).correlation
print(f"Spearman Correlation for Random Forest: {spearman_corr_rf:.1%}")

Spearman Correlation for Random Forest: 72.4%


In [64]:
# Get feature importances
feature_importances = rf_model.feature_importances_

# Get feature names
feature_names = x_train.columns

# Combine feature names and feature importances and print
feature_importance_list = list(zip(feature_names, feature_importances))
feature_importance_list.sort(key=lambda x: x[1], reverse=True)  # Sort by importance (descending)

# Print feature importance
for feature, importance in feature_importance_list:
    print(f"{feature}: {importance:.4f}")

TARGET_gb: 0.3117
TARGET_adb: 0.3085
TARGET_xgb: 0.3018
TARGET_ma: 0.0434
stacked_prediction: 0.0346


In [65]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor()
gb_model.fit(x_train, y_train['TARGET'])
y_pred_gb = gb_model.predict(x_test)
spearman_corr_gb = spearmanr(y_pred_gb, y_test).correlation
print(f"Spearman Correlation for Gradient Boosting: {spearman_corr_gb:.1%}")


Spearman Correlation for Gradient Boosting: 75.1%


In [66]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor()
xgb_model.fit(x_train, y_train['TARGET'])
y_pred_xgb = xgb_model.predict(x_test)
spearman_corr_xgb = spearmanr(y_pred_xgb, y_test).correlation
print(f"Spearman Correlation for XGBoost: {spearman_corr_xgb:.1%}")

Spearman Correlation for XGBoost: 65.1%


In [67]:
pred = svm_model.predict(X_test)
df_test = pd.read_csv('../challenge_data/X_test.csv')
test_pred = pd.DataFrame()
test_pred['ID'] = df_test['ID']
test_pred['TARGET'] = pred
test_pred.to_csv('./Submission/' + 'stacking_final' + '.csv', index=False)